In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import sys
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from functools import reduce, partial
from timeit import default_timer

from src.utils.utils import *
from src.models.base import FNO1d
from src.models.single_step import BOON_FNO1d

In [3]:
torch.manual_seed(0)
np.random.seed(0)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dirichlet

In [8]:
ntrain = 500
ntest = 100

sub = 1 #subsampling rate
h = 500 // sub #total grid size divided by the subsampling rate

s = h

batch_size = 20
learning_rate = 0.001

epochs = 500
step_size = 50
gamma = 0.5

modes = 16
width = 64

In [9]:
################################################################
# read data
################################################################

# LOAD YOUR DATA HERE consists of pairs (a,u) with randomly generated initial conditions or PDE parameters a
# and solution u
rw_ = loadmat('Data/1D/Burgers Dirichlet/burgers_Dirichlet_1D_data_nu_0_point_1.mat')

x_data = rw_['a'].astype(np.float32) # shape (num_random_simulations, number_grid_points)
y_data = rw_['u'].astype(np.float32) # shape (num_random_simulations, number_grid_points)

x_data = torch.from_numpy(x_data)
y_data = torch.from_numpy(y_data)

x_train = x_data[:ntrain,:]
y_train = y_data[:ntrain,:]
x_test = x_data[-ntest:,:]
y_test = y_data[-ntest:,:]

x_train = x_train.unsqueeze(-1)
x_test = x_test.unsqueeze(-1)

left_bdry_train = y_train[:,0].unsqueeze(-1)
right_bdry_train = y_train[:,-1].unsqueeze(-1)

left_bdry_test = y_test[:,0].unsqueeze(-1)
right_bdry_test = y_test[:,-1].unsqueeze(-1)

In [11]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train, left_bdry_train, right_bdry_train), 
                                            batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test, left_bdry_test, right_bdry_test), 
                                            batch_size=batch_size, shuffle=False)

In [13]:
base_no = FNO1d(modes, width)
model = BOON_FNO1d(width, 
                    base_no,
                    bdy_type='dirichlet').to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)

for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    for batch in train_loader:
        x, y, left, right = batch
        x, y, left, right = x.to(device), y.to(device), left.to(device), right.to(device)
        
        optimizer.zero_grad()
        out = model(x, 
                    bdy_left={'val':left}, 
                    bdy_right={'val':right}
                   )

        l2 = myloss(out.view(batch_size, -1), y.view(batch_size, -1))
        l2.backward() # use the l2 relative loss

        optimizer.step()
        train_l2 += l2.item()
        

    scheduler.step()
    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for batch in test_loader:
            x, y, left, right = batch
            x, y, left, right = x.to(device), y.to(device), left.to(device), right.to(device)

            out = model(x, 
                        bdy_left={'val':left}, 
                        bdy_right={'val':right}
                       )

            test_l2 += myloss(out.view(batch_size, -1), y.view(batch_size, -1)).item()

    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1,  train_l2, test_l2)

# Neumann

In [8]:
################################################################
#  configurations
################################################################
ntrain = 500
ntest = 100

sub = 1 #subsampling rate
h =500 // sub #total grid size divided by the subsampling rate

s = h
N = h

batch_size = 20
learning_rate = 0.001

epochs = 500
step_size = 50
gamma = 0.5

modes = 16
width = 64

In [9]:
################################################################
# read data
################################################################
# LOAD YOUR DATA HERE consists of pairs (a,u) with randomly generated initial conditions or PDE parameters a
# and solution u
rw_ = loadmat('Data/1D/Heat Neumann/heat_Neumann_1D.mat')

x_data = rw_['a'].astype(np.float32) # shape (num_random_simulations, number_grid_points)
y_data = rw_['u'].astype(np.float32) # shape (num_random_simulations, number_grid_points)

x_data = torch.from_numpy(x_data)
y_data = torch.from_numpy(y_data)

x_train = x_data[:ntrain,::sub]
y_train = y_data[:ntrain,::sub]
x_test = x_data[-ntest:,::sub]
y_test = y_data[-ntest:,::sub]

x_train = x_train.unsqueeze(-1)
x_test = x_test.unsqueeze(-1)

In [11]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), 
                                            batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), 
                                            batch_size=batch_size, shuffle=False)

In [13]:
h = 1/(N-1)

coeffs_finite_difference_right = np.array([-1/(3*h), 3/(2*h), -3/h , 11/(6*h)])
coeffs_finite_difference_right = coeffs_finite_difference_right.astype(np.float32)

normalized_coeff_right = np.array([-2/11, 9/11, -18/11, 6*h/11]).astype(np.float32)
normalized_coeff_right = torch.from_numpy(normalized_coeff_right).to(device)

diff_fn_right = partial(compute_finite_diff, normalized_coeff_right, loc=-1)

coeffs_finite_difference_left = np.array([1, -1])
coeffs_finite_difference_left = coeffs_finite_difference_left.astype(np.float32)

normalized_coeff_left = np.array([1, -1]).astype(np.float32)
normalized_coeff_left = torch.from_numpy(normalized_coeff_left).to(device)
diff_fn_left = partial(compute_finite_diff, normalized_coeff_left, loc=0)

neumann_bdy_left = 0.0

U = np.array([5])
U = torch.from_numpy(U).to(device)

t = np.array([0.5])
t = torch.from_numpy(t).to(device)
neumann_bdy_right = U*torch.sin(torch.pi*t)

In [14]:
base_no = FNO1d(modes, width)
model = BOON_FNO1d(width, 
                    base_no,
                    bdy_type='neumann').to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)

for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    for batch in train_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        out = model(x, 
                    bdy_left={'val':neumann_bdy_left, 'diff_fn':diff_fn_left},
                    bdy_right={'val':neumann_bdy_right, 'diff_fn':diff_fn_right},
                   )

        l2 = myloss(out.view(batch_size, -1), y.view(batch_size, -1))
        l2.backward() # use the l2 relative loss

        optimizer.step()
        train_l2 += l2.item()
        

    scheduler.step()
    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for batch in test_loader:
            x, y = batch
            x, y = x.to(device), y.to(device)

            out = model(x, 
                        bdy_left={'val':neumann_bdy_left, 'diff_fn':diff_fn_left},
                        bdy_right={'val':neumann_bdy_right, 'diff_fn':diff_fn_right},
                       )

            test_l2 += myloss(out.view(batch_size, -1), y.view(batch_size, -1)).item()

    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1, train_l2, test_l2)

# Periodic

In [9]:
################################################################
#  configurations
################################################################
ntrain = 500
ntest = 100

sub = 2**6 #subsampling rate
h = 2**13 // sub #total grid size divided by the subsampling rate

s = h

batch_size = 20
learning_rate = 0.001

epochs = 500
step_size = 50
gamma = 0.5

modes = 16
width = 64

In [10]:
################################################################
# read data
################################################################
# LOAD YOUR DATA HERE consists of pairs (a,u) with randomly generated initial conditions or PDE parameters a
# and solution u
rw_ = loadmat('Data/1D/Burgers Periodic/burgers_data_R10.mat')

x_data = rw_['a'].astype(np.float32) # shape (num_random_simulations, number_grid_points)
y_data = rw_['u'].astype(np.float32) # shape (num_random_simulations, number_grid_points)

x_data = torch.from_numpy(x_data)
y_data = torch.from_numpy(y_data)

x_train = x_data[:ntrain,::sub]
y_train = y_data[:ntrain,::sub]
x_test = x_data[-ntest:,::sub]
y_test = y_data[-ntest:,::sub]

# strictly put periodic bdy condition
x_train[:, -1] = x_train[:, 0]
y_train[:, -1] = y_train[:, 0]
x_test[:, -1] = x_test[:, 0]
y_test[:, -1] = y_test[:, 0]

x_train = x_train.unsqueeze(-1)
x_test = x_test.unsqueeze(-1)

In [12]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), 
                                            batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), 
                                            batch_size=batch_size, shuffle=False)

In [14]:
base_no = FNO1d(modes, width)
model = BOON_FNO1d(width, 
                    base_no,
                    bdy_type='periodic').to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)

for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    for batch in train_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        out = model(x)

        l2 = myloss(out.view(batch_size, -1), y.view(batch_size, -1))
        l2.backward() # use the l2 relative loss

        optimizer.step()
        train_l2 += l2.item()
        

    scheduler.step()
    model.eval()
    test_l2 = 0.0
    test_l2_near_bdy = 0.0
    with torch.no_grad():
        for batch in test_loader:
            x, y = batch
            x, y = x.to(device), y.to(device)
            out = model(x)
            test_l2 += myloss(out.view(batch_size, -1), y.view(batch_size, -1)).item()

    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1, train_l2, test_l2)